In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.SVHN(root='./data', split='train', download=True, transform=transform)
test_dataset = torchvision.datasets.SVHN(root='./data', split='test', download=True, transform=transform)

train_dataset = torch.utils.data.Subset(train_dataset, range(0, len(train_dataset)//4))
test_dataset = torch.utils.data.Subset(test_dataset, range(0, len(test_dataset)//4))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

models = {
    "LeNet-5": torchvision.models.googlenet(pretrained=True),
    "VGG": torchvision.models.vgg11(pretrained=True),
    "ResNet18": torchvision.models.resnet18(pretrained=True),
    "ResNet50": torchvision.models.resnet50(pretrained=True),
    "ResNet101": torchvision.models.resnet101(pretrained=True),
    "AlexNet": torchvision.models.alexnet(pretrained=True)
}

for model_name, model in models.items():
    if isinstance(model, torchvision.models.AlexNet):
        num_ftrs = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(num_ftrs, 10)
    elif isinstance(model, torchvision.models.VGG):
        num_ftrs = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(num_ftrs, 10)
    else:
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, 10)

def train(model, train_loader, test_loader, epochs=1):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print(f"Accuracy on test set: {accuracy}")

for model_name, model in models.items():
    print(f"\nTraining {model_name}")
    train(model, train_loader, test_loader)


Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat

Training LeNet-5
Epoch 1/1, Loss: 1.9775714762119465
Accuracy on test set: 0.46711739397664415

Training VGG
Epoch 1/1, Loss: 1.1054078442294424
Accuracy on test set: 0.8116164720344192

Training ResNet18
Epoch 1/1, Loss: 1.4150035256708124
Accuracy on test set: 0.7323294406883836

Training ResNet50
